In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [5]:
# Stap 2: Inlezen en voorbereiden van de data
# Lees de CSV-bestand in (vervang 'path/to/your/dataset.csv' met de daadwerkelijke pad naar je CSV)
df = pd.read_csv('data.csv', sep=';')

# Verken de data (optioneel)
print(df.head())  # Print de eerste paar regels van de dataset
print(df.info())  # Geef een overzicht van de dataset

# Selecteer de features (X) en de target variabele (y)
X = df.drop(['HeartDisease' ,'CaseNumber', 'LastName', 'PostCode'], axis=1)  # Verwijder niet-relevante kolommen
y = df['HeartDisease']

# Converteer categorische kolommen naar numeriek (One-Hot Encoding voor categorische features)
X = pd.get_dummies(X, drop_first=True)

# Splits de data in een train- en testset (80% trainen, 20% testen)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    CaseNumber  LastName  PostCode  Age Sex ChestPainType  RestingBP  \
0  431-06-4243   Richard     99847   40   M           ATA        140   
1  415-39-7809  Sheppard     64192   49   F           NAP        160   
2  517-18-4618    Howard     29132   37   M           ATA        130   
3  634-33-8726    Taylor     12930   48   F           ASY        138   
4  151-40-1619   Mcgrath     79393   54   M           NAP        150   

   Cholesterol  FastingBS  RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  \
0          289          0           0    172              N        0       Up   
1          180          0           0    156              N       10     Flat   
2          283          0           1     98              N        0       Up   
3          214          0           0    108              Y       15     Flat   
4          195          0           0    122              N        0       Up   

   HeartDisease  
0             0  
1             1  
2             0  
3       

In [6]:
# Stap 3: Model Selectie en Training
# Definieer de modellen die je wilt proberen
models = {
    "Linear Regression": LinearRegression(),  # Voorbeeldmodel, gebruik voor vergelijking, niet echt voor classificatie
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Ridge Classifier": RidgeClassifier(),
    "Lasso Regression": Lasso(),  # Voorbeeldmodel, gebruik voor vergelijking, niet echt voor classificatie
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
}

# Train elk model en evalueer de prestaties
results = {}
for name, model in models.items():
    try:
        model.fit(X_train, y_train)  # Train het model
        
        # Check of predict_proba beschikbaar is, anders gebruik predict direct voor ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            # Voor lineaire regressie en lasso, cast naar binair resultaat
            y_pred_proba = model.predict(X_test)
            y_pred_proba = (y_pred_proba >= 0.5).astype(int)  # Simuleer classificatie output
        
        y_pred = model.predict(X_test)  # Voorspel met de testdata
        accuracy = accuracy_score(y_test, y_pred)  # Bereken de nauwkeurigheid
        roc_auc = roc_auc_score(y_test, y_pred_proba)  # Bereken de ROC AUC
        f1 = f1_score(y_test, y_pred)  # Bereken de F1-score
        
        results[name] = {'Accuracy': accuracy, 'ROC AUC': roc_auc, 'F1 Score': f1}
        # Print de evaluatie metrics
        print(f"{name} - Accuracy: {accuracy:.4f}, ROC AUC: {roc_auc:.4f}, F1 Score: {f1:.4f}")
    
    except Exception as e:
        print(f"Error training {name}: {e}")

# Bepaal welk model de beste prestaties levert op basis van ROC AUC
best_model = max(results, key=lambda x: results[x]['ROC AUC'])
print(f"\nBest model based on ROC AUC: {best_model}")

Error training Linear Regression: Classification metrics can't handle a mix of binary and continuous targets
Logistic Regression - Accuracy: 0.8525, ROC AUC: 0.9113, F1 Score: 0.8789
Ridge Classifier - Accuracy: 0.8470, ROC AUC: 0.8389, F1 Score: 0.8750
Error training Lasso Regression: Classification metrics can't handle a mix of binary and continuous targets
Decision Tree - Accuracy: 0.8033, ROC AUC: 0.7955, F1 Score: 0.8378
Random Forest - Accuracy: 0.8579, ROC AUC: 0.9149, F1 Score: 0.8870
Gradient Boosting - Accuracy: 0.8525, ROC AUC: 0.9233, F1 Score: 0.8831

Best model based on ROC AUC: Gradient Boosting
